# add nlp part idx

## configs

In [1]:
LANG = 'port'
PART = 0

## dir setup

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

this_file_path = os.path.dirname(os.path.abspath("__file__"))
# if script (not notebook)...
# project_dir = os.path.join(os.path.dirname(__file__), os.pardir)

# project directory
project_dir = os.path.join(this_file_path, os.pardir)

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [3]:
raw_data_dir = os.path.join(project_dir, os.environ.get("RAW_DATA_DIR"))
processed_data_dir = os.path.join(project_dir, os.environ.get("PROCESSED_DATA_DIR"))
interim_data_dir = os.path.join(project_dir, os.environ.get("INTERIM_DATA_DIR"))
wordvecs_data_dir = os.path.join(project_dir, os.environ.get("WORDVECS_DATA_DIR"))
figures_dir = os.path.join(project_dir, os.environ.get("FIGURES_DIR"))
reports_dir = os.path.join(project_dir, os.environ.get("REPORTS_DIR"))
cv_dir = os.path.join(project_dir, os.environ.get("CV_DIR"))
models_dir = os.path.join(project_dir, os.environ.get("MODELS_DIR"))

## imports

In [4]:
import numpy as np
from datetime import datetime

In [5]:
import pandas as pd
# pd.set_option('max_rows', 100)
# pd.set_option('max_columns', None)
# pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
from tqdm import tqdm
tqdm.pandas()

/Users/hitoshinagano/.virtualenvs/tensorflow2/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
# import scipy.sparse as sparse
# import implicit

In [8]:
# from sklearn.metrics import ndcg_score

In [9]:
# from sklearn.model_selection import ParameterGrid

In [10]:
# import mlflow

In [11]:
# import pickle

In [12]:
import spacy

In [13]:
os.path.join(wordvecs_data_dir, 'nilc50skip')

'/Users/hitoshinagano/Documents/MercadoLibre_competicao/notebooks/../data/wordvecs/nilc50skip'

In [14]:
port_nlp_fn = 'nilc50skip'
port_nlp_fp = os.path.join(wordvecs_data_dir, port_nlp_fn) 

port_nlp = spacy.load(port_nlp_fp)
# espa_nlp = spacy.load('es_core_news_md')

if LANG == 'port':
    nlp = port_nlp
elif LANG == 'espa':
    nlp = espa_nlp
else:
    assert False

# import pt_core_news_md
# port_nlp = pt_core_news_md.load()

## project imports

In [15]:
import sys
sys.path.insert(0, '..')

In [16]:
from src.features import *
from src.models import *

* remove buy events: no additional relevance attributed to buy
* remove event timestamp: no additional relevance attributed to recency
* remove search events: no additional relevance attributed to search
* aggregates counts - higher relevance attributed to more views

# exploration

In [17]:
# train_fn = 'train_dataset.pkl'
# test_fn = 'test_dataset.pkl'
# train, test = read_processed(train_fn, test_fn)

In [18]:
# true_fn = 'true.pkl'
# true_fp = os.path.join(processed_data_dir, true_fn)
# true_df = pd.read_pickle(true_fp)

# item_data_fn = 'item_data.jl.gz'
# item_data = pd.read_json(os.path.join(raw_data_dir, item_data_fn), lines=True)
# item_domain = item_data[['item_id', 'domain_id']]

#### create nlp objects for titles in item_data

In [19]:
# item_data['title_lower'] = item_data.title.str.lower()
# item_data_unique = item_data.title_lower.drop_duplicates().to_frame()

In [20]:
part_fn = 'item_data_unique_part' + str(PART) + '.pkl'
part_fp = os.path.join(interim_data_dir, part_fn)
item_data_unique = pd.read_pickle(part_fp)

In [22]:
def emb_vectors(x):
    v = nlp(x)
    return (v.vector / v.vector_norm).astype(np.float16)

item_data_unique[LANG] = item_data_unique.title_lower.progress_apply(emb_vectors)

In [23]:
part_fn = 'item_data_unique_part_emb_' + str(PART) + '.pkl'
part_fp = os.path.join(interim_data_dir, part_fn)
item_data_unique.to_pickle(part_fp)

In [24]:
item_data_unique

,title_lower,port
0,casa sola en venta con gran patio solo pago de...,"[0.1279, 0.02943, 0.10016, 0.0578, -0.1276, -0..."
1,resident evil origins collection nintendo swit...,"[0.0374, -0.2139, 0.2559, -0.09314, -0.1992, 0..."
2,falda de imitación piel negra,"[-0.02623, 0.03317, -0.0725, -0.06097, -0.031,..."
3,powercolor red devil radeon rx 580 8gb gddr5,"[0.129, -0.07404, 0.2448, -0.06235, -0.06555, ..."
4,laptop hp nx6320 core duo con puerto db9 windo...,"[0.1315, -0.1361, 0.2778, -0.0952, -0.1708, 0...."
...,...,...
496579,fone bluetooth u8 power bank 2200ma,"[0.0701, -0.0475, 0.1593, -0.09827, -0.1565, 0..."
496580,bateria jt40 smartphone moto g6 plus xt1926 jt 40,"[0.0591, -0.1248, 0.2115, -0.06143, -0.2146, 0..."
496581,peugeot 206 1.6presence flex 4p,"[0.1562, -0.09924, 0.3923, -0.03232, -0.1863, ..."
496582,extensor ajustador de calça para gestante com ...,"[-0.2253, 0.03046, -0.002068, -0.1687, 0.03296..."


______

In [70]:
embs = item_data_unique.port.apply(pd.Series)

In [109]:
i = item_data_unique.sample()
idx = i.index
i

,title_lower,port
1437704,teclado philco phn14 cce mp-07g38pa-36051 (tc18),"[0.041961487, 0.014135246, 0.10670626, -0.1545..."


In [110]:
top = (embs @ embs.loc[idx[0]]).sort_values(ascending = False).head(20).index
item_data_unique.loc[top]

,title_lower,port
1437704,teclado philco phn14 cce mp-07g38pa-36051 (tc18),"[0.041961487, 0.014135246, 0.10670626, -0.1545..."
245290,kit 3 soportes motor/transmisión volkswagen vo...,"[0.09904308, 0.035100315, 0.25201547, -0.18363..."
1847448,manche g force plus (funciona também como vola...,"[-0.0040776273, -0.06546781, 0.100338474, -0.1..."
628987,filtro esportivo k&n 69-1013ts honda civic 1.8...,"[0.11240914, -0.030196754, 0.24186185, -0.1417..."
903407,wireless adaptador 900mbps sem fio lan b/g/n a...,"[0.18258348, 0.012308925, 0.13747403, -0.18767..."
372346,elegoo uno r3 atmel atmega328p (tipo arduino),"[0.017379548, 0.001265869, 0.15671021, -0.0939..."
1127112,kit carenagem cg 150 titan ks 2010 preto c/ade...,"[0.07493867, 0.045194387, 0.21444468, -0.24976..."
577303,kit 2 x bateria talkabout t200/t400/mj motor...,"[0.1405283, -0.059899174, 0.19933334, -0.16570..."
1342187,roda gm montana sport kr r26 / aro 15x6 / prat...,"[0.24035543, 0.10233009, 0.1314769, -0.259539,..."
362616,"parafusadeira bosch go bivolt 3,6v c/maleta + ...","[0.12379752, 0.001000576, 0.2003301, -0.138254..."


In [45]:
item_data_unique.loc[1618842, 'port']

0.70614177

In [46]:
item_data_unique.loc[656375, 'port'] @ item_data_unique.loc[1778179, 'port']

0.5465207

In [38]:
one_title = item_data_unique.sample().title_lower.values[0]
v = nlp(one_title)

In [40]:
v.vector_norm

1.407821071056158

In [24]:
from numpy import linalg as LA

In [25]:
LA.norm(v.vector.astype(np.float16))

0.994

In [26]:
v.vector_norm

0.9943244670224373